In [1]:
import random
import httpx
import msgpack
import threading
import time
import os
import argparse
import json
import scipy
import numpy as np
from sklearn.preprocessing import normalize
from tqdm import tqdm
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize

In [24]:
!pip install jsonlines

Defaulting to user installation because normal site-packages is not writeable


In [2]:
gpt_2_api = 'http://0.0.0.0:6006/inference'
gpt2_french = 'http://0.0.0.0:6007/inference'
gptj_french = 'http://0.0.0.0:6008/inference'
llama_api_french = 'http://0.0.0.0:6009/inference'

In [4]:
import json
import glob

def merge_jsonl(input_files, output_file):
    combined_data = []
    for file in input_files:
        with open(file, 'r') as f:
            for line in f:
                json_data = json.loads(line.strip())
                combined_data.append(json_data)

    with open(output_file, 'w') as f:
        for item in combined_data:
            f.write(json.dumps(item) + '\n')

# Example usage
input_files = glob.glob('./SeqXGPT-Bench/*.jsonl')  # List of JSONL files in current directory
output_file = 'input.jsonl'

merge_jsonl(input_files, output_file)

In [4]:
!pip install googletrans==4.0.0-rc1
!pip install googletrans==3.1.0a0
!pip install google_trans_new

Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.1.0a0
    Uninstalling googletrans-3.1.0a0:
      Successfully uninstalled googletrans-3.1.0a0
Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-3.1.0a0-py3-none-any.whl
  Attempting uninstall: googletrans
    Found existing installation: googletrans 4.0.0rc1
    Uninstalling googletrans-4.0.0rc1:
      Successfully uninstalled googletrans-4.0.0rc1
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import googletrans
from googletrans import Translator
import jsonlines

# see available languages with the below
def translate(input_file,output_file,lan):
    # print(googletrans.LANGUAGES)

    # Init
    translator = Translator()
    
    input_json = []
    input_json_text = []
    
    with open(input_file, 'r') as file:
        for line in file:
            data = json.loads(line)
            input_json.append(data)
            input_json_text.append(data['text'])
    
    batches = []
    batch_size = 100
    for i in range(0,len(input_json_text), batch_size):
        batch = input_json_text[i:i + batch_size]
        batches.append(batch)     
    
    input_json_text = []
    
    for index,batch in enumerate(batches):
        outputs = translator.translate(batch, src='en', dest=lan)
        input_json_text.extend(outputs)
        print('batch '+str(index)+' done, batches left'+str(len(batches)-1-index))
            
    
    for index,inputs in enumerate(input_json):
        inputs['text'] = input_json_text[index].text
            
    with jsonlines.open(output_file+'_'+lan+'.jsonl', mode='w') as writer:
        writer.write_all(input_json)
                

In [ ]:
import threading
t1 = threading.Thread(target=translate, args=('input.jsonl','input_lang','ar'))
t2 = threading.Thread(target=translate, args=('input.jsonl','input_lang','fr'))

t1.start()
t2.start()

t1.join()
t2.join()

batch 0 done, batches left359
batch 0 done, batches left359
batch 1 done, batches left358
batch 2 done, batches left357
batch 1 done, batches left358
batch 3 done, batches left356
batch 4 done, batches left355
batch 2 done, batches left357
batch 5 done, batches left354
batch 6 done, batches left353
batch 3 done, batches left356
batch 7 done, batches left352
batch 8 done, batches left351
batch 4 done, batches left355
batch 9 done, batches left350
batch 5 done, batches left354
batch 10 done, batches left349
batch 11 done, batches left348
batch 6 done, batches left353
batch 12 done, batches left347
batch 13 done, batches left346
batch 7 done, batches left352
batch 14 done, batches left345
batch 8 done, batches left351
batch 15 done, batches left344
batch 16 done, batches left343
batch 9 done, batches left350
batch 17 done, batches left342
batch 18 done, batches left341
batch 10 done, batches left349
batch 19 done, batches left340
batch 11 done, batches left348
batch 20 done, batches left3

In [4]:
import jsonlines
# get human line
def getHumanSentences(input_file,output_file):
    input_json = []
    with open(input_file, 'r') as file:
        for line in file:
            data = json.loads(line)
            if data['label'] == 'human':
                input_json.append(data)
    
    with jsonlines.open(output_file, mode='w') as writer:
        writer.write_all(input_json)

In [3]:
getHumanSentences('input_lang_zh-cn.jsonl','cn_human_lines.jsonl')

In [5]:
# get 12000 of all lang
import jsonlines
def getsubset(num,input_file,output_file):
    labels_count_map = {
        'gpt2': 0,
        'gptneo': 0,
        'gptj': 0,
        'llama': 0,
        'gpt3re': 0,
        'human': 0,
    }
    
    input_json = []
    with open(input_file, 'r') as file:
        for line in file:
            data = json.loads(line)
            if labels_count_map[data['label']] < num:
                labels_count_map[data['label']] += 1
                input_json.append(data)

            
    with jsonlines.open(output_file, mode='w') as writer:
        writer.write_all(input_json)

In [11]:
getsubset(1000,'input_lang_hi.jsonl','input_lang_hi_1000.jsonl')
# getsubset(2000,'input_lang_fr.jsonl','input_lang_fr_small.jsonl')
# getsubset(2000,'input_lang_ar.jsonl','input_lang_ar_small.jsonl')
# getsubset(2000,'input_lang_zh-cn.jsonl','input_lang_zh-cn_small.jsonl')

In [3]:
def access_api(text, api_url, do_generate=False):
    """

    :param text: input text
    :param api_url: api
    :param do_generate: whether generate or not
    :return:
    """
    with httpx.Client(timeout=None) as client:
        post_data = {
            "text": text,
            "do_generate": do_generate,
        }
        prediction = client.post(api_url,
                                 data=msgpack.packb(post_data),
                                 timeout=None)
    if prediction.status_code == 200:
        content = msgpack.unpackb(prediction.content)
    else:
        content = None
    return content

In [4]:
def get_features(input_file, output_file, model_names, model_apis):
    """
    get [losses, begin_idx_list, ll_tokens_list, label_int, label] based on raw lines
    """

    en_labels = {
        'gpt2': 0,
        'gptneo': 1,
        'gptj': 1,
        'llama': 2,
        'gpt3re': 3,
        'gpt3sum': 3,
        'human': 4,
        'alpaca': None,
        'dolly': None,
    }


    with open(input_file, 'r') as f:
        lines = [json.loads(line) for line in f]

    print('input file:{}, length:{}'.format(input_file, len(lines)))

    with open(output_file, 'w', encoding='utf-8') as f:
        for data in tqdm(lines):
            line = data['text']
            # Split the text into sentences
            sentences = re.split(r'[。！？]', line)

            # Remove empty strings
            sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
            
            label = data['label']
            
            if label == 'human':
                prompt_len = 0
            else:
                prompt_len = len(sentences[0])

            losses = data['losses'] if 'losses' in data else []
            begin_idx_list = data['begin_idx_list'] if 'begin_idx_list' in data else []
            ll_tokens_list = data['ll_tokens_list'] if 'll_tokens_list' in data else []
            
            label_dict = en_labels

            label_int = label_dict[label]

            error_flag = False
            for api in model_apis:
                try:
                    loss, begin_word_idx, ll_tokens = access_api(line, api)
                except TypeError:
                    print("return NoneType, probably gpu OOM, discard this sample")
                    error_flag = True
                    break
                losses.append(loss)
                begin_idx_list.append(begin_word_idx)
                ll_tokens_list.append(ll_tokens)
            # if oom, discard this sample
            if error_flag:
                continue

            result = {
                'losses': losses,
                'begin_idx_list': begin_idx_list,
                'll_tokens_list': ll_tokens_list,
                'label_int': label_int,
                'label': label,
                'text': line,
                'prompt_len': prompt_len
            }

            f.write(json.dumps(result, ensure_ascii=False) + '\n')

In [ ]:
get_features('input_lang_fr.jsonl','output_lang_fr.jsonl',['gpt_2','gpt2_french'],[gpt_2_api,gpt2_french])

input file:input_lang_fr.jsonl, length:35904


 51%|█████▏    | 18401/35904 [2:00:43<2:12:53,  2.20it/s]

In [ ]:
get_features('input_lang_fr.jsonl','output_lang_fr_gptj.jsonl',['gptj_french'],[gptj_french])

input file:input_lang_fr.jsonl, length:35904


 19%|█▉        | 6830/35904 [38:20<2:36:28,  3.10it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 41%|████      | 14550/35904 [1:22:18<1:48:22,  3.28it/s]

In [ ]:
get_features('input_lang_fr.jsonl','output_lang_fr_llama.jsonl',['llama_api_french'],[llama_api_french])

In [7]:
 total = 0;
with open('output_lang_fr_llama.jsonl', 'r') as file:
    for line in file:
        total = total + 1

print(total)

35904


In [9]:
import jsonlines

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)


input_jsonl = read_jsonl('output_prompt_fr_merged35K.jsonl')
input_jsonl2 = read_jsonl('output_lang_fr_llama.jsonl')
# input_jsonl3 = read_jsonl('output_lang_fr_llama.jsonl')

i=0
text = []
i = 0
j = 0

while i < len(input_jsonl):
    if input_jsonl[i]['text'] == input_jsonl2[j]['text']:
        input_jsonl[i]['losses'].extend(input_jsonl2[j]['losses'])
        input_jsonl[i]['begin_idx_list'].extend(input_jsonl2[j]['begin_idx_list'])
        input_jsonl[i]['ll_tokens_list'].extend(input_jsonl2[j]['ll_tokens_list'])
        i = i + 1
        j = j + 1
    else:
        text.append(input_jsonl2[j])
        j = j+1


print(len(text))
write_jsonl('output_prompt_fr_merged35K.jsonl',input_jsonl)

1


In [18]:
import jsonlines

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)


input_jsonl = read_jsonl('output_lang_zh-cn_new.jsonl')

i=0
for item in input_jsonl:
    if len(item['losses']) < 2:
        i=i+1
print(i)

0


In [9]:
import jsonlines

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)


input_jsonl = read_jsonl('output_prompt_zh-cn_merged35K.jsonl')
input_jsonl2 = read_jsonl('output_lang_zh-cn_llama.jsonl')


i=0
for item1 in input_jsonl:
    if i==0:
        print(len(item1['begin_idx_list']))
    i=i+1


4


In [11]:
import jsonlines
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize, word_tokenize

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)


input_jsonl = read_jsonl('output_prompt_fr_merged35K.jsonl')

for item1 in input_jsonl:
    if item1['label'] != 'human' :
        sent_separator = nltk.data.load('tokenizers/punkt/french.pickle')
        sents = sent_separator.tokenize(item1['text'])
        item1['prompt_len'] = len(sents[0])


write_jsonl('output_prompt_fr_merged35K.jsonl',input_jsonl)


[nltk_data] Downloading package punkt to /home/pmukher/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import jsonlines

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)


input_jsonl = read_jsonl('output_all_zh-cn_small.jsonl')
output_jsonl = read_jsonl('output_all_zh-cn_final.jsonl')

i = 0
for item1 in input_jsonl:
    if output_jsonl[i]['text'] == item1['text']:
        if 'prompt_len' in item1 :
            output_jsonl[i]['prompt_len'] = item1['prompt_len']
        else:
            output_jsonl[i]['prompt_len'] = 0
        i = i + 1

write_jsonl('output_prompt_zh-cn_merged.jsonl',output_jsonl)

In [38]:
# data binary
import jsonlines

from collections import Counter

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

binary_data = read_jsonl('output_prompt_zh-cn_merged35K.jsonl')

i=0
labels = []
for item in binary_data:
    if len(item['losses'])==2:
        i = i+1
        labels.append(item['label'])
        
element_counts = Counter(labels)

# Print the counts
for element, count in element_counts.items():
    print(f"{element}: {count}")

In [12]:
# data binary
import jsonlines

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

binary_data = read_jsonl('output_prompt_fr_merged35K.jsonl')

for item in binary_data:
    if item['label'] != 'human':
        item['label'] = 'AI'

write_jsonl('output_binary_merged_fr.jsonl',binary_data)

In [4]:
from langdetect import detect

def detect_language(sentence):
    return detect(sentence)

# Example usage
english_sentence = "This is an English sentence."
chinese_sentence = "今年三月，政府宣布了任命一名东安格利亚市长的计划，但遭到多个议会的拒绝。据BBC了解，现在地方议会和政府的领导人已经提出了新的建议。但该计划仍需获得剑桥郡、诺福克郡和萨福克郡所有 23 个议会的批准。该计划还需要得到剑桥郡的批准\n无能领导者的危险\n我相信所有商界领袖都应该至少每周思考并做出以下承诺，因为这不仅对公司的健康至关重要，而且对经济乃至整个国家都至关重要。此外，需要注意的是，每位领导者必须代表每位团队成员做出承诺，反之亦然。换句话说，如果组织中的所有 200 人都不参与，则领导者也不参与。我坚信，我们作为一个经济体的成功、我们的生活质量，甚至我们作为一个国家的安全和保障，都在很大程度上取决于我们每个人尽最大努力确保我们组织中的每个团队成员都在做对我们国家来说可能是最成功的工作。"

print(detect_language(english_sentence))  # Output: en (English)
print(detect_language(chinese_sentence)) 

en
zh-cn


In [2]:
!pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=22a6b10ea42be676da991bd34db262400663aacf474919d7f586d5bbc1368bfe
  Stored in directory: /home/pmukher/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [2]:
import jsonlines
import re

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

binary_data = read_jsonl('output_binary_merged_zh-cn.jsonl')

for item in binary_data:
    line = item['text']
    # Split the text into sentences
    sentences = re.split(r'[。！？]', line)

    item['prompt_len'] = len(sentences[0])
        
write_jsonl('output_binary_merged_zh-cn.jsonl',binary_data)

In [6]:
from langdetect import detect
def detect_language(sentence):
    return detect(sentence)

In [12]:
import jsonlines
import re

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

binary_data = read_jsonl('output_binary_merged_zh-cn.jsonl')

with open('output_binary_merged_zh-cn_new.jsonl', 'w', encoding='utf-8') as f:
    for item in binary_data:
        if detect_language(item['text']) == 'en':
            i = i + 1
        else:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')


In [2]:
import jsonlines
import re

def read_jsonl(file_path):
    with jsonlines.open(file_path, 'r') as reader:
        return list(reader)
    
def write_jsonl(file_path, data):
    with jsonlines.open(file_path, 'w') as writer:
        writer.write_all(data)

binary_data = read_jsonl('output_binary_merged_zh-cn_new.jsonl')

print(len(binary_data))

35743
